# Create H, A, T

In [1]:
import pandas as pd
df = pd.read_csv('./datasets/raw/tdec.csv')

In [22]:
dfs = df.sample(100, random_state=99)

In [3]:
import stanza

nlp = stanza.Pipeline(
    lang="id",
    processors="tokenize,pos,lemma",
    download_method=stanza.DownloadMethod.REUSE_RESOURCES,
)

d:\repo\lda-bertopic\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-19 12:47:06 WARNING: Language id package default expects mwt, which has been added
2023-08-19 12:47:07 INFO: Loading these models for language: id (Indonesian):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| pos       | gsd     |
| lemma     | gsd     |

2023-08-19 12:47:07 INFO: Using device: cpu
2023-08-19 12:47:07 INFO: Loading: tokenize
2023-08-19 12:47:07 INFO: Loading: mwt
2023-08-19 12:47:07 INFO: Loading: pos
2023-08-19 12:47:07 INFO: Loading: lemma
2023-08-19 12:47:07 INFO: Done loading processors!


In [4]:
from time import time

_HAS_t = {}

t_start = time()
H = dfs.filter(['headline'], axis=1)
H.columns = ['H']

_HAS_t['H'] = time() - t_start
A = dfs.filter(['body'], axis=1)
A.columns = ['A']
A['A'] = dfs['headline'] + '. ' + A['A']

_HAS_t['A'] = time() - _HAS_t['H']
S = dfs.filter(['body'], axis=1)
S.columns = ['S']
S['S'] = dfs['headline'] + '. ' + S['S']
in_S = list(S['S'].apply(lambda h: stanza.Document([], text=h)))
out_S = nlp(in_S)
S = pd.DataFrame([s.text for a in out_S for s in a.sentences], columns=['S'])

_HAS_t['S'] = time() - _HAS_t['A']

In [5]:
from utils import e_variant

HAS_t = pd.DataFrame(pd.Series(e_variant()), columns=['variant'])

d:\repo\lda-bertopic\.venv\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
d:\repo\lda-bertopic\.venv\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
d:\repo\lda-bertopic\.venv\lib\site-packages\umap\distances.py:1086: NumbaDeprecationWarning: The 'nopython'

In [6]:
r = []
for v in HAS_t['variant']:
    r.append(_HAS_t[v[0]])
HAS_t['segmentation'] = pd.Series(r)

In [7]:
HAS_t

,variant,segmentation
0,HT,5.998611e-03
1,HCL,5.998611e-03
2,HCLW,5.998611e-03
3,HCLWN,5.998611e-03
4,AT,1.692424e+09
5,ACL,1.692424e+09
6,ACLW,1.692424e+09
7,ACLWN,1.692424e+09
8,ST,4.204551e+01
9,SCL,4.204551e+01


In [8]:
HAS = pd.concat([H, A, S], axis=1)
# HAS = pd.concat([H, A], axis=1)

In [9]:
HAS

,H,A,S
3584,"Punya Bonus Demografi, Airlangga Hartarto: Per...","Punya Bonus Demografi, Airlangga Hartarto: Per...",NaN
6705,"Menjelang Natal, Harga Emas Antam Turun Rp 5 R...","Menjelang Natal, Harga Emas Antam Turun Rp 5 R...",NaN
5520,"Pemilih Anies Baswedan Condong Dukung PKS, Nas...","Pemilih Anies Baswedan Condong Dukung PKS, Nas...",NaN
1653,"Soal KUHP Baru, Komnas HAM Nilai DPR Belum Ber...","Soal KUHP Baru, Komnas HAM Nilai DPR Belum Ber...","Menyikapi hal tersebut, Wakil Ketua KPK itu pu..."
2268,BWF World Tour Finals 2022: Jonatan Christie A...,BWF World Tour Finals 2022: Jonatan Christie A...,NaN
...,...,...,...
1835,NaN,NaN,Pergerakan angin ke timur itu karena ditarik o...
1836,NaN,NaN,“Potensi bahaya ini masih sangat besar di hari...
1837,NaN,NaN,Simak breaking news dan berita pilihan dari Te...
1838,NaN,NaN,Klik https://t.me/tempodotcoupdate untuk berga...


In [10]:
HAS_t.to_parquet('./results/HAS_t.parquet')
HAS.to_parquet('./results/HAS.parquet')

# Create Variants

In [11]:
import pandas as pd

HAS = pd.read_parquet('./results/HAS.parquet')
HAS_t = pd.read_parquet('./results/HAS_t.parquet')

In [12]:
import stanza

nlp = stanza.Pipeline(
    lang="id",
    processors="tokenize,pos,lemma",
    download_method=stanza.DownloadMethod.REUSE_RESOURCES,
)

2023-08-19 12:48:01 WARNING: Language id package default expects mwt, which has been added
2023-08-19 12:48:01 INFO: Loading these models for language: id (Indonesian):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| pos       | gsd     |
| lemma     | gsd     |

2023-08-19 12:48:01 INFO: Using device: cpu
2023-08-19 12:48:01 INFO: Loading: tokenize
2023-08-19 12:48:01 INFO: Loading: mwt
2023-08-19 12:48:01 INFO: Loading: pos
2023-08-19 12:48:01 INFO: Loading: lemma
2023-08-19 12:48:01 INFO: Done loading processors!


In [13]:
from gensim.models.phrases import Phrases

allowed_pos = ['NOUN', 'PROPN', 'VERB', 'X']

def preprocess_clw(out_df, c=False, l=False, w=False, n=False):
    r = [] # result
    if n:
        r_t = []
    for d in out_df:
        tokens = []
        for s in d.sentences:
            for token in s.words:
                _token = token.text
                if w:
                    if token.upos not in allowed_pos:
                        continue
                if l:
                    _token = token.lemma if token.lemma else _token
                if c:
                    _token = _token if l else token.text.lower()
                tokens.append(_token)
        r.append(' '.join(tokens))
        if n:
            r_t.append(tokens)
    if n:
        bigram = Phrases(r_t).freeze()
        trigram = Phrases(bigram[r_t]).freeze()
        r = [' '.join(d) for d in trigram[bigram[r_t]]]
    return r

In [14]:
from time import time
from utils import e_variant


def preprocess():
    t_start = time()
    nlp_datasets = {}
    for v in e_variant():
        df = HAS[v[0]].dropna()
        in_doc = list(df.apply(lambda x: stanza.Document([], text=x)))
        out_doc = nlp(in_doc)
        nlp_datasets[v[0]] = out_doc
    t_base = time() - t_start
    r = {} # result
    r_time = []
    variant = e_variant()
    for v in variant:
        t_preprocess = time()
        process_C = 'C' in v
        process_L = 'L' in v
        process_W = 'W' in v
        process_N = 'N' in v
        r[v] = pd.Series(preprocess_clw(
            nlp_datasets[v[0]],
            c=process_C,
            l=process_L,
            w=process_W,
            n=process_N
        ))
        r_time.append(time() - t_preprocess + t_base)
    HAS_t['preprocess'] = pd.Series(r_time)
    return pd.DataFrame(r)

In [15]:
HAS_p = preprocess()

In [16]:
HAS_p

,HT,HCL,HCLW,HCLWN,AT,ACL,ACLW,ACLWN,ST,SCL,SCLW,SCLWN
0,"Punya Bonus Demografi , Airlangga Hartarto : P...","punya bonus demografi , airlangga hartarto : p...",punya bonus demografi airlangga hartarto perta...,punya bonus demografi airlangga hartarto perta...,"Punya Bonus Demografi , Airlangga Hartarto : P...","punya bonus demografi , airlangga hartarto : p...",punya bonus demografi airlangga hartarto perta...,punya bonus_demografi airlangga hartarto perta...,"Menyikapi hal tersebut , Wakil Ketua KPK itu p...","sikap hal tersebut , wakil ketua kpk itu pun i...",sikap hal wakil ketua kpk imbau jajar asn pemp...,sikap hal wakil_ketua kpk imbau jajar asn pemp...
1,"Menjelang Natal , Harga Emas Antam Turun Rp 5 ...","menjelang natal , harga emas antam turun rp 5 ...",natal harga emas antam turun rp ribu jadi,natal harga emas antam turun rp ribu jadi,"Menjelang Natal , Harga Emas Antam Turun Rp 5 ...","menjelang natal , harga emas antam turun rp 5 ...",natal harga emas antam turun rp ribu jadi nata...,natal harga emas antam turun rp ribu jadi nata...,Turnamen tersebut akan berlangsung di Thailand...,"turnamen tersebut akan langsung di thailand , ...",turnamen langsung thailand desember,turnamen langsung thailand desember
2,"Pemilih Anies Baswedan Condong Dukung PKS , Na...","penpilih anies baswedan condong dukung pks , n...",penpilih anies baswedan condong dukung pks nas...,penpilih anies baswedan condong dukung pks nas...,"Pemilih Anies Baswedan Condong Dukung PKS , Na...","penpilih anies baswedan condong dukung pks , n...",penpilih anies baswedan condong dukung pks nas...,penpilih anies_baswedan condong dukung pks nas...,NABILA RAMADHANTY PUTRI DARMADI | HINDUSTAN TI...,nabila ramadhanty putri darmadi | hindustan ti...,nabila ramadhanty putri darmadi hindustan time...,nabila ramadhanty putri darmadi hindustan time...
3,"Soal KUHP Baru , Komnas HAM Nilai DPR Belum Be...","soal kuhp baru , komnas ham nilai dpr belum sp...",soal kuhp baru komnas ham nilai dpr spektif ham,soal kuhp baru komnas ham nilai dpr spektif ham,"Soal KUHP Baru , Komnas HAM Nilai DPR Belum Be...","soal kuhp baru , komnas ham nilai dpr belum sp...",soal kuhp baru komnas ham nilai dpr spektif ha...,soal kuhp baru komnas_ham nilai dpr spektif ha...,Mantan pemain Arsenal itu menuturkan alasan ke...,mantan main arsenal itu tutur alasan datang di...,mantan main arsenal tutur alasan datang indone...,mantan main arsenal tutur alasan datang indone...
4,BWF World Tour Finals 2022 : Jonatan Christie ...,bwf world tour finals 2022 : jonatan christie ...,bwf world tour finals jonatan christie lolos s...,bwf world tour finals jonatan christie lolos s...,BWF World Tour Finals 2022 : Jonatan Christie ...,bwf world tour finals 2022 : jonatan christie ...,bwf world tour finals jonatan christie lolos s...,bwf_world tour_finals jonatan christie lolos s...,"jumlah nya mencapai 200 juta euro per musim , ...","jumlah dia capai 200 juta euro per musim , ter...",jumlah capai euro musim termasuk gaji janji la...,jumlah capai euro musim termasuk gaji janji la...
...,...,...,...,...,...,...,...,...,...,...,...,...
1835,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pergerakan angin ke timur itu karena ditarik o...,gerak angin ke timur itu karena tarik oleh pus...,gerak angin timur tarik pusat tekan sebut trop...,gerak angin timur tarik pusat tekan sebut trop...
1836,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,“ Potensi bahaya ini masih sangat besar di har...,“ potensi bahaya ini masih sangat besar di har...,potensi bahaya hari ada tunjuk adanya luruh ka...,potensi bahaya hari ada tunjuk adanya luruh ka...
1837,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Simak breaking news dan berita pilihan dari Te...,simak breaking news dan berita pilih dari temp...,simak breaking news berita pilih tempo.co kana...,simak_breaking_news_berita pilih_tempo.co_kana...
1838,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Klik https://t.me/tempodotcoupdate untuk berga...,klik https://t.me/tempodotcoupdate untuk gabung .,klik gabung,klik_gabung


In [17]:
HAS_t

,variant,segmentation,preprocess
0,HT,5.998611e-03,422.800828
1,HCL,5.998611e-03,422.799833
2,HCLW,5.998611e-03,422.800833
3,HCLWN,5.998611e-03,422.812833
4,AT,1.692424e+09,422.819834
5,ACL,1.692424e+09,422.831834
6,ACLW,1.692424e+09,422.838855
7,ACLWN,1.692424e+09,423.086812
8,ST,4.204551e+01,422.824833
9,SCL,4.204551e+01,422.838843


In [18]:
HAS_p.to_parquet('./results/HAS_p.parquet')
HAS_t.to_parquet('./results/HAS_t.parquet')